In [21]:
# # +++++++++++++ old approach +++++++++++++
# # +++++++++++++ DELETE LATER +++++++++++++

# # duplicate frames so we can fill up the missing values in the dataframe

# delme = RFID_df.copy(deep=True)
# len(delme.index)

# # print(list(delme[delme.IdRFID =='041917923E'].Frame))
# # print(list(delme[delme.IdRFID =='041917C16C'].Frame))
# # print(list(delme[delme.IdRFID =='041917C87E'].Frame))

# frames_041917C87E = list(delme[delme.IdRFID =='041917C87E'].Frame)
# print(frames_041917C87E)

# diff_frames_041917C87E = np.diff(frames_041917C87E)
# print(diff_frames_041917C87E)
# print(diff_frames_041917C87E.sum())

# df_fill = pd.DataFrame(columns=delme.columns)

# for frame, diff_frame in zip(frames_041917C87E, diff_frames_041917C87E):
#     id_to_append = delme[(delme['IdRFID'] == '041917C87E') & (delme['Frame'] == frame)]
#     delme = delme.append([id_to_append] * diff_frame, ignore_index=True)

#     delme = delme.sort_values(by=['IdRFID', 'Frame'])
# delme = delme.reset_index() 
# delme

# i = 0
# def increment_frame(col):
#     if col == col.shift(-1):
#         i += 1
#         return col + i
#     else: 
#         return col

# delme_ordered = (delme
#     .groupby('IdRFID')
#     .apply(
#         lambda x: x.assign(
#             # New_frame_nr = x['Frame'] + i if x['Frame'] == x['Frame'].shift(-1) else x['Frame']       
#             New_frame_nr = increment_frame(x['Frame'])
#         )
#     )
# )

# delme_ordered

In [23]:
# Euclidean distance between two RFID readers

RFID_df_from_to = RFID_df.copy()

# def calc_distance (RFID_from, RFID_to):
#     a = RFID_df_from_to_coords[RFID_df_from_to_coords.RFID_reader_to == RFID_from].iloc[0]
#     b = RFID_df_from_to_coords[RFID_df_from_to_coords.RFID_reader_to == RFID_to].iloc[0]
#     dist = ((b.x_to - a.x_to)**2 + (b.y_to - a.y_to)**2)
#     return dist
# calc_distance('R1.4', 'R1.3') # test

# Shift the row of the RFID readers by 1 for each mouse. 
if ('DateTime' in RFID_df_from_to):
    RFID_df_from_to = (RFID_df_from_to
        .sort_values(by=['DateTime'])
        .groupby('IdRFID')
        .apply(
            lambda x: x.assign(
                # Frame = x['Frame'].shift(-1), # shift the frame so that the framenr. corresponds to the `to` coords
                Mouse_ID = x['IdRFID'],
                DateTime_in = x['DateTime'].dt.strftime("%Y-%m-%d %H:%M:%S.%f"),
                DateTime_out = x['DateTime'].shift(-1).dt.strftime("%Y-%m-%d %H:%M:%S.%f"),
                Duration_sec = (x['DateTime'].shift(-1) - x['DateTime']).dt.total_seconds(), #.round(2),
                RFID_reader_from = x['unitLabel'],
                RFID_reader_to = x['unitLabel'].shift(-1),
                senseRFIDrecords = x['senseRFIDrecords'],
                eventDuration =  x['eventDuration']
                # end_time = x['DateTime'].dt.strftime("%Y-%m-%d %H:%M:%S.%f") + dt.timedelta(seconds = x['eventDuration'] /1000)
                
                
                # Distance = map(calc_distance, x['unitlabel'], x['unitlabel'].shift(-1))
            )
        )    
        .drop(columns=['DateTime', 'IdRFID', 'unitLabel'], errors='ignore') # drop old cols
        .dropna() 
    )
# RFID_df_from_to['Distance'] = list(map(calc_distance, RFID_df_from_to['RFID_reader_from'], RFID_df_from_to['RFID_reader_to'])) # add distance between the readers

try:
    # merge RFID_df_from_to_coords with RFID_df_from_to on coords (x, y) to create plots
    # Important!: the coords of the `x_to & x_to` should be used and not `x_from & y_from` because
    # the RFID reader software write the detection in the csv file when the reader is logged out (mouse is logged out from reader)
    # e.g. m1 is detected at ts 0 in R1.1 and stays there for 20s, then nothing will be written in the csv file. When the mouse change its position 
    RFID_df_from_to = RFID_df_from_to.merge(RFID_df_coords.rename(columns={'RFID_reader' : 'RFID_reader_from'}), how="inner", left_on="RFID_reader_from", right_on="RFID_reader_from", suffixes=(False, False))

    # RFID_df_from_to_coords.rename(columns = {'RFID_reader_to':'RFID_reader_from'}, inplace=True)
    # RFID_df_from_to = RFID_df_from_to.merge(RFID_df_from_to_coords, how="inner", left_on="RFID_reader_from", right_on="RFID_reader_from", suffixes=(False, False))
    # RFID_df_from_to_coords.rename(columns = {'RFID_reader_from':'RFID_reader_to'}, inplace=True)
except:
    # avoid duplicating columns when run multile times
    pass

# sort the RFID dataframe by the date in
RFID_df_from_to = RFID_df_from_to.sort_values(by=['DateTime_in'], ignore_index=True)

RFID_df_from_to.head()

,Frame,eventDuration,senseRFIDrecords,Mouse_ID,DateTime_in,DateTime_out,Duration_sec,RFID_reader_from,RFID_reader_to,x,y
0,16,0.0,1.0,041917C4F5,2022-05-25 14:05:23.019003,2022-05-25 14:05:23.374003,0.355000,R1.2,R1.3,526,241
1,24,497.0,2.0,041917C4F5,2022-05-25 14:05:23.374003,2022-05-25 14:05:31.350001,7.975998,R1.3,R2.2,728,241
2,136,1101.0,5.0,041917C16C,2022-05-25 14:05:27.829003,2022-05-25 14:05:30.131001,2.301998,R2.4,R2.3,923,486
3,193,0.0,1.0,041917C16C,2022-05-25 14:05:30.131001,2022-05-25 14:05:31.672999,1.541998,R2.3,R2.3,728,486
4,224,0.0,1.0,041917C4F5,2022-05-25 14:05:31.350001,2022-05-25 14:05:31.605003,0.255002,R2.2,R2.1,526,486


In [28]:

try:
    # merge RFID_df_coords with RFID_df on coords (x, y)
    RFID_df = RFID_df.merge(RFID_df_coords[['RFID_reader', 'area_coords']], how="inner", left_on="RFID_reader", right_on="RFID_reader", suffixes=(False, False))

except:
    # avoid duplicating columns if running multile times
    pass

# # sort the RFID dataframe by the date in
RFID_df = RFID_df.sort_values(by=['DateTime'], ignore_index=True)

RFID_df.head()

RFID_df_coords

,RFID_reader,x,y,area_coords
0,R1.1,338,241,"((0, 0), (431, 359))"
1,R1.2,526,241,"((432, 0), (626, 359))"
2,R1.3,728,241,"((627, 0), (824, 359))"
3,R1.4,923,241,"((825, 0), (1279, 359))"
4,R2.1,338,486,"((0, 360), (431, 1279))"
5,R2.2,526,486,"((432, 360), (626, 1279))"
6,R2.3,728,486,"((627, 360), (824, 1279))"
7,R2.4,923,486,"((825, 360), (1279, 1279))"


c:\Users\Mohamad\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\Users\Mohamad\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\Users\Mohamad\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

,Frame,Mouse_ID,x,y,Reader
0,16,041917C4F5,728.0,241.0,R1.3
1,17,041917C4F5,728.0,241.0,R1.3
2,18,041917C4F5,728.0,241.0,R1.3
3,19,041917C4F5,728.0,241.0,R1.3
4,20,041917C4F5,728.0,241.0,R1.3
...,...,...,...,...,...
8407,2932,041917923E,728.0,241.0,R1.3
8408,2933,041917923E,728.0,241.0,R1.3
8409,2934,041917923E,728.0,241.0,R1.3
8410,2935,041917923E,728.0,241.0,R1.3


# Matching last approac

## plot last approach so that dlc and rfid match have the same color

## Last approach: swaping..

### plot after correction


# END OF LAST APPROACH

# Plot (RFID)

# Read DLC HDF5 file

In [51]:
def mice_id_swap(mouse):
    for both_names in RFID_DLC_mice_id:
        if mouse == both_names[0]:
            return both_names[1]
        elif mouse == both_names[1]:
            return both_names[0]
    print('Wrong mice name format!. No swap will occur!')
    return mouse

# PLot RFID annotations on video

# Match RFID and DLC mice

In [64]:
## IF this or does not work, then it is because of replacing -1 with nan for no overlap cases

# USE THIS LATER FOR MULTI INDEXING!!!!
df = {k: pd.DataFrame(v) for k,v in mice_overlap_freq.items()}
df = pd.concat(df, axis=1)
# df.columns = df.columns.droplevel(3)
df

# df = pd.DataFrame.from_dict(mice_overlap_freq, orient="index").stack().to_frame()
# # to break out the lists into columns
# df = pd.DataFrame(df[0].values.tolist(), index=df.index).T
# df




041917C4F5         041917923E         041917C16C        
        mouse3             mouse2             mouse1        
     frequency overlap  frequency overlap  frequency overlap
0          1.0    -1.0          1      -1        1.0    -1.0
1          1.0     0.0          1       1        1.0     1.0
2          1.0     1.0          1       0        1.0     0.0
3          1.0     0.0          1       1        1.0     1.0
4          1.0     1.0          1       0        1.0     0.0
..         ...     ...        ...     ...        ...     ...
623        NaN     NaN          1       1        NaN     NaN
624        NaN     NaN          1       0        NaN     NaN
625        NaN     NaN          4       1        NaN     NaN
626        NaN     NaN          3       0        NaN     NaN
627        NaN     NaN          1       1        NaN     NaN

[628 rows x 6 columns]

# Correct detections

In [71]:
# dlc_coords_dist_skipped_nan

# New approach for correction of annotations

1. Identify switching useing a threshold based on distance between annotations between frame i and i+1
2. Go to the frame where the switching occurs then calc distance of each keypoint of mouse j to all readers. Then assign the mouse to the closes RFID detection.


In [74]:
# for each frame
    # calc distance between frame i and i-1 (i-x where x a frame with no Nan coords)
    # if distance > threshold:
        # correct_distance (calc distance between rfid and dlc and ssign value to min distance)

In [77]:
def calc_distance_between_RFID_DLC(DLC, RFID_dict):
    '''
    Takes a dlc row for a mouse and bodypart (ts, x, y,likelihood)
    and a RFID data as a list of dictionary for specific frame. Each dict has the mouse id and its (x, y) coords.

    return mouse id with its RFID coords which is the closest one to the DLC detection 
    '''
    id = math.nan
    dist = math.inf
    coords = math.nan
    same_reader_position = []

    for dict in RFID_dict:
    
        RFID_x = list(dict.values())[0][0]
        RFID_y = list(dict.values())[0][1]
        tmp_dist = math.sqrt((DLC.x - RFID_x)**2 + (DLC.y - RFID_y)**2)

        # collect same distances
        if tmp_dist == dist:
            same_reader_position.append(tmp_dist)
    
        # pick only min distances
        if tmp_dist < dist:
            id = list(dict.keys())[0]
            dist = tmp_dist
            coords = (RFID_x, RFID_y)

    # if the minimum distance to a reader exists twice then return nan because correction is impossible
    if dist in same_reader_position:
            id = math.nan
            dist = math.inf
            coords = math.nan
        
    return id, coords

# dlc_distances = list(map(calc_distance_between_RFID_DLC, dlc_hdf[scorer, 'mouse3', 'snout'].iloc[0], RFID_df.iloc[[0]]))
calc_distance_between_RFID_DLC(dlc_hdf[scorer, 'mouse1', 'spine4'].iloc[650], filled_RFID_dict[650])

# for each frame
    # calc distance between frame i and i-1 (i-x where x a frame with no Nan coords)
    # if distance > threshold:
        # correct_distance (calc distance between rfid and dlc and ssign value to min distance)
        

c:\Users\Mohamad\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


('041917C4F5', (728, 241))

In [78]:
RFID_DLC_mice_id

Index([('041917C4F5', 'mouse3'), ('041917923E', 'mouse2'),
       ('041917C16C', 'mouse1')],
      dtype='object')

In [79]:
def swap_coords(dlc_hdf_copy, frame, mouse1,  mouse2, bodypart):
    """
    Swap two coords of the same bodypart between two mice 
    """

    
    tmp_x = dlc_hdf_copy.loc[frame, (scorer, mouse1, bodypart, 'x')]
    tmp_y = dlc_hdf_copy.loc[frame, (scorer, mouse1, bodypart, 'y')]

    dlc_hdf_copy.loc[frame, (scorer, mouse1, bodypart, 'x')] = dlc_hdf_copy.loc[frame, (scorer, mouse2, bodypart, 'x')]
    dlc_hdf_copy.loc[frame, (scorer, mouse1, bodypart, 'y')] = dlc_hdf_copy.loc[frame, (scorer, mouse2, bodypart, 'y')]
    
    dlc_hdf_copy.loc[frame, (scorer, mouse2, bodypart, 'x')] = tmp_x
    dlc_hdf_copy.loc[frame, (scorer, mouse2, bodypart, 'y')] = tmp_y


In [77]:
dlc_coords_dist_skipped_nan = {}
threshold = 300
dlc_hdf_copy = dlc_hdf.copy()
for mouse in dlc_mice_id:
    for bodypart in bodyparts:
        print(mouse, bodypart)
        dlc_coords_dist_skipped_nan[mouse, bodypart, 'distance'] = []
        dlc_coords_dist_skipped_nan[mouse, bodypart, 'frame'] = []

        mouse_coords_dict = {}
        
        f = 0
        while f < len(dlc_hdf_copy):
            if not pd.isna(dlc_hdf_copy[scorer, mouse, bodypart, 'x'].iloc[f]):
                # print(f)
                x = dlc_hdf_copy[scorer, mouse, bodypart, 'x'].iloc[f]
                y = dlc_hdf_copy[scorer, mouse, bodypart, 'y'].iloc[f]

                f_shift = f + 1
                while f_shift < len(dlc_hdf_copy):
                    if not pd.isna(dlc_hdf_copy[scorer, mouse, bodypart, 'x'].iloc[f_shift]):        
                        
                        x_shift = dlc_hdf_copy[scorer, mouse, bodypart, 'x'].iloc[f_shift]
                        y_shift = dlc_hdf_copy[scorer, mouse, bodypart, 'y'].iloc[f_shift]

                        dist = math.sqrt((x - x_shift)**2 + (y - y_shift)**2) 
                        
                        if dist > threshold:
                            from_to = '{}-{}'.format(f, f_shift)
                            dlc_coords_dist_skipped_nan[mouse, bodypart, 'distance'].append(dist)
                            dlc_coords_dist_skipped_nan[mouse, bodypart, 'frame'].append(from_to)

                            # find the min distance between the DLC-misslabeled coords for a bodypart and all other RFID readers
                            new_id, _ = calc_distance_between_RFID_DLC(dlc_hdf_copy[scorer, mouse, bodypart].iloc[f_shift], filled_RFID_dict[f_shift])
                            
                            # swap coords to do the correction
                            if (new_id is not math.nan):
                                print('forward swapping..')
                                swap_coords(dlc_hdf_copy, f_shift, mouse, mice_id_swap(new_id), bodypart)

                                
                                # correct the current frame as well.
                                # Since this frame will be nan value, it will be skipped automaticly in the next iteration.
                                # Now we check backwords until we reach the last frame without nan value (which is f) and we calculatre the distance bewteen f_shift+1 (=f+2) and last not nan value (f).
                                # We update the coords in fshift2 if we achieve the threshold
                                
                                # f_shift_2 = f_shift+1

                                # while f_shift_2 < len(dlc_hdf_copy):
                                #     if not pd.isna(dlc_hdf_copy[scorer, mouse, bodypart, 'x'].iloc[f_shift_2]):        
                                    
                                #         x_shift_2 = dlc_hdf_copy[scorer, mouse, bodypart, 'x'].iloc[f_shift_2]
                                #         y_shift_2 = dlc_hdf_copy[scorer, mouse, bodypart, 'y'].iloc[f_shift_2]


                                #         dist = math.sqrt((x - x_shift_2)**2 + (y - y_shift_2)**2) 
                                        
                                #         if dist > threshold:
                                #             # from_to = '{}-{}'.format(f, f_shift)
                                #             # dlc_coords_dist_skipped_nan[mouse, bodypart, 'distance'].append(dist)
                                #             # dlc_coords_dist_skipped_nan[mouse, bodypart, 'frame'].append(from_to)

                                #             # find the min distance between the DLC-misslabeled coords for a bodypart and all other RFID readers
                                #             new_id, _ = calc_distance_between_RFID_DLC(dlc_hdf_copy[scorer, mouse, bodypart].iloc[f_shift_2], filled_RFID_dict[f_shift_2])
                                            
                                #             # swap coords to do the correction
                                #             if (new_id is not math.nan):
                                #                 swap_coords(dlc_hdf_copy, f_shift_2, mouse, mice_id_swap(new_id), bodypart)

                                #         else:
                                #             break

                                   
                                #     f_shift_2 += 1
                                        
                        break



                    else:
                        f_shift += 1

                f += 1

            else:
                f += 1

mouse1 leftEar


c:\Users\Mohamad\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
mouse1 rightEar
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swapping..
forward swappin

In [78]:
RFID_DLC_mice_id

Index([('041917C4F5', 'mouse3'), ('041917923E', 'mouse2'),
       ('041917C16C', 'mouse1')],
      dtype='object')

In [79]:
dlc_coords_dist_skipped_nan
dlc_coords_dist_skipped_nan_fromDictToDF(dlc_coords_dist_skipped_nan)

mouse1                                                           \
       leftEar               rightEar               shoulder              
      distance      frame    distance      frame    distance      frame   
0   607.064142    636-651  580.338590    634-651  560.387118    636-651   
1   599.235349    653-657  599.991212    653-657  555.648501    653-659   
2   413.526775  1579-1598  449.575976  1579-1598  449.964439  1579-1598   
3   410.971260  1599-1627  446.071322  1599-1627  450.856001  1599-1627   
4   607.339881  1909-1913  514.087012  1902-1913  554.195784  1909-1914   
5   489.515186  1917-1918  516.329787  1917-1918  471.309647  1917-1918   
6          NaN        NaN  308.582759  1918-1919         NaN        NaN   
7          NaN        NaN  315.108919  1919-1920         NaN        NaN   
8          NaN        NaN  324.258492  1920-1921         NaN        NaN   
9          NaN        NaN  328.797201  1921-1922         NaN        NaN   
10         NaN        NaN  335.096110  1922-1923         NaN        NaN   
11         NaN        NaN  346.241416  1923-1924         NaN        NaN   
12         NaN        NaN  348.935046  1924-1925         NaN        NaN   
13         NaN        NaN  346.312106  1925-1926         NaN        NaN   
14         NaN        NaN  348.677153  1926-1927         NaN        NaN   
15         NaN        NaN  354.788349  1927-1928         NaN        NaN   
16         NaN        NaN  347.389504  1928-1929         NaN        NaN   
17         NaN        NaN  353.093320  1929-1930         NaN        NaN   
18         NaN        NaN  352.061248  1930-1931         NaN        NaN   
19         NaN        NaN  349.535249  1931-1932         NaN        NaN   
20         NaN        NaN  348.036794  1932-1933         NaN        NaN   
21         NaN        NaN  348.752375  1933-1934         NaN        NaN   
22         NaN        NaN  350.648161  1934-1935         NaN        NaN   
23         NaN        NaN  341.647379  1935-1936         NaN        NaN   
24         NaN        NaN  340.433917  1936-1937         NaN        NaN   
25         NaN        NaN  335.524153  1937-1938         NaN        NaN   
26         NaN        NaN  334.281755  1938-1939         NaN        NaN   
27         NaN        NaN  338.023207  1939-1940         NaN        NaN   
28         NaN        NaN  328.962272  1940-1941         NaN        NaN   
29         NaN        NaN  327.088554  1941-1942         NaN        NaN   
30         NaN        NaN  332.024625  1942-1943         NaN        NaN   
31         NaN        NaN  322.012879  1943-1944         NaN        NaN   
32         NaN        NaN  317.610232  1944-1945         NaN        NaN   
33         NaN        NaN  312.151999  1945-1946         NaN        NaN   
34         NaN        NaN  307.914422  1946-1947         NaN        NaN   
35         NaN        NaN  312.872783  1947-1948         NaN        NaN   
36         NaN        NaN  316.027093  1948-1949         NaN        NaN   
37         NaN        NaN  320.214208  1949-1950         NaN        NaN   
38         NaN        NaN  323.504348  1950-1951         NaN        NaN   
39         NaN        NaN  320.443403  1951-1952         NaN        NaN   
40         NaN        NaN  319.451229  1952-1953         NaN        NaN   
41         NaN        NaN  315.202462  1953-1954         NaN        NaN   
42         NaN        NaN  314.341630  1954-1955         NaN        NaN   
43         NaN        NaN  320.764504  1955-1956         NaN        NaN   
44         NaN        NaN  325.595214  1956-1957         NaN        NaN   
45         NaN        NaN  327.525727  1957-1958         NaN        NaN   
46         NaN        NaN  329.616546  1958-1959         NaN        NaN   
47         NaN        NaN  331.983356  1959-1960         NaN        NaN   
48         NaN        NaN  332.591638  1960-1961         NaN        NaN   
49         NaN        NaN  330.584511  1961-1962         NaN        NaN   
50         NaN        NaN  335.27

In [66]:
RFID_DLC_mice_id

Index([('041917C4F5', 'mouse3'), ('041917923E', 'mouse2'),
       ('041917C16C', 'mouse1')],
      dtype='object')

-----------------------------------------------------------------

Annotate more frames and train more to get no identity swapping:
https://forum.image.sc/t/madlc-identity-switching/58260

.. Also, since you are using a multi-animal model, if you haven’t already make sure you update DLC from github (now 2.2.0.2 - just yesterday), and use the DLCrnet multi-stage model, which really improves multi-animal tracking and speed:
https://forum.image.sc/t/identity-switching-when-4-mice-tracking-using-madlc/57589

In [88]:
def find_corresponding_overlap(frame, dlc_hdf, dlc_id, dlc_coords, rfid_list):
    '''
    Find the corresponding rfid tag name for a mouse in a specific frame of a given dlc coord in the same frame number.
    The algorithm can be improved by using IOU algorithm (intersection over union). Or also we can use the length of the sceletion instaead of defining bounding boxes.
    '''

    corr_rfid_id = []
    for id_with_coords in rfid_list:
    
        rfid_pos = list(id_with_coords.values())[0]

        # check if there is an overlap between a specific bodypart and an rfid reader
        if check_overlap((dlc_coords.x, dlc_coords.y), (rfid_pos[0], rfid_pos[1])) == 1:
            corr_rfid_id.append(list(id_with_coords.keys())[0])


    if len(corr_rfid_id) == 0:
        # print('No overlap is found.')
        return 0

    elif len(corr_rfid_id) == 1:
        # print('RFID overlap is found.')
        
        # check if there are other mice in the same rfid range
        for another_mouse in RFID_mice_id:
            if another_mouse != dlc_id:
                for bp in bodyparts:
                    another_mouse_coords =  dlc_hdf[scorer, another_mouse, bp].iloc[frame]                    
                    # Do not continue calculating if there is more thatn one dlc mouse in the same rfid range
                    if check_overlap((dlc_coords.x, dlc_coords.y), (another_mouse_coords.x, another_mouse_coords.y)) == 1:
                        # print('But there are other mice in the same RFID range --> No swapping yet..')
                        return -2
        
        return corr_rfid_id[0]

    # Ensure that we do  not have two rfid reads in the same rfid range.
    else :
        # print('Multiple overlaps are found. Cannot assign dlc to rfid..')
        return -1

# coords = dlc_hdf_copy[scorer, '041917C16C', 'spine4'].iloc[650]
# find_corresponding_overlap(650, dlc_hdf_copy, '041917C16C', coords,  filled_RFID_dict[650])
    

In [250]:
crucial_bp = ['spine3', 'spine2', 'spine1', 'shoulder', 'spine4']

for f in range(670):
    print(f)
    if f >= min(filled_RFID_dict.keys()):
        dlc_coords = math.nan
        for current_id in RFID_mice_id:
            for bp in crucial_bp:
                dlc_coords = dlc_hdf_copy[scorer, current_id, bp].iloc[f]
                if not math.isnan(dlc_coords.x):
                    break                

            # Calculate the binary overlap and find a match.
            new_id = find_corresponding_overlap(f, dlc_hdf_copy, current_id, dlc_coords, filled_RFID_dict[f])

            if type(new_id) is str:
                # Correct and swap the mouse id if the already assigned rfid does not match with the new one.
                if new_id != current_id:
                    print('======== Swapping', current_id, 'to', new_id, 'at frame:', f, '========')
                    swap_coords(dlc_hdf_copy, f, new_id, current_id, bp)

        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29

c:\Users\Mohamad\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
c:\Users\Mohamad\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2947: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)



30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
2

# Playground

In [72]:
# import h5py
# f.close()
# h5py_path = r"X:\Social-Interaction\Tracking\results-recording_RP_2021_12_15--14-00-27_cut_first_min\after_analyzing\h5.h5"
# f = h5py.File(h5py_path,'a')


# for frm in range(0, 1500): # note that frame numer starts by 0 in teh dlc hdf file!!!
#     dlc_mouse_coords = f_HDFStore.get("tracks").get(dlc_hdf_info['scorer'][0]).get(dlc_hdf_info['individuals'][m]).get('spine4')
#     dlc_mouse_coords['area_coords'] = dlc_mouse_coords.apply(lambda x: calc_RFID_area(x), axis=1)

#     dlc_mouse_coords['area_coords'] = dlc_mouse_coords.apply(lambda x: calc_RFID_area(x), axis=1)



In [73]:
# pickle_file_path_name = r"X:\Social-Interaction\Tracking\results-recording_RP_2021_12_15--14-00-27_cut_first_min\after_convert_detections2tracklets\recording_RP_2021_12_15--14-00-27_cut_first_minDLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500_el.pickle"
# pickle_df = pd.read_pickle(pickle_file_path_name)
# pickle_df

In [74]:
# d = {"frame_nr": list(range(1,1501))}
# frame_nr_df = pd.DataFrame(data=d)
# frame_nr_df

In [75]:
# f_HDFStore = pd.HDFStore(h5_file_path_name)

In [76]:
# f_HDFStore.groups()

In [77]:
# f_HDFStore.put("frame", df, format="table", data_columns=True)
# f_HDFStore["/tracks"]

In [78]:
# f_HDFStore["/tracks"].keys()[23]

In [79]:
# len(f_HDFStore["/tracks"].columns.levels)

In [80]:
# # This apporach works also using `HDFStore`

# # temp path to the DLC hdf file
# h5_file_path_name = r"X:\Social-Interaction\Tracking\results-recording_RP_2021_12_15--14-00-27_cut_first_min\after_analyzing\h5.h5"

# # read hdf file which is produced from DeepLabCut (DLC).
# f_HDFStore = pd.HDFStore(h5_file_path_name)

# # create a dictionary which holds some information about the DLC hdf file
# # Keyes: level names: e.g. scorer , individuals, keypoints, coords
# # Values: lavel values: e.g. DLC project name (as scorer), mouse1, snout, x
# dlc_hdf_info = {}

# # length of tge level of the DLC hdf file (=4)
# lvl_depth = len(f_HDFStore["/tracks"].columns.levels)

# # assign values to the `dlc_hdf_info` dictionary. See the dictionary decleration description for more information
# for i in range(0, lvl_depth):
#     dlc_hdf_lvl_name = f_HDFStore["/tracks"].columns.levels[i].name
#     dlc_hdf_col_name = f_HDFStore["/tracks"].columns.levels[i]
#     dlc_hdf_info[dlc_hdf_lvl_name] = dlc_hdf_col_name
    
# # get the number of mice in the dlc hdf file
# dlc_nr_mice = len(dlc_hdf_info['individuals'])

# # # extract mice coords from DLC hdf file for each mouse and save the result as a new dataframe (mouse id wiht coords) 
# # for m in range(0, dlc_nr_mice):
# #     dlc_mouse_coords = f_HDFStore.get("tracks").get(dlc_hdf_info['scorer'][0]).get(dlc_hdf_info['individuals'][m]).get('spine4')
# #     dlc_mouse_coords['area_coords'] = dlc_mouse_coords.apply(lambda x: calc_RFID_area(x), axis=1)
# # dlc_mouse_coords

In [81]:
dlc_hdf['DLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500', 'mouse1', 'snout']
# demle

coords,likelihood,x,y
0,1.0,1007.841,332.785
1,1.0,993.048,344.234
2,1.0,990.657,352.971
3,1.0,991.630,365.491
4,1.0,989.520,372.641
...,...,...,...
2877,1.0,450.883,548.781
2878,1.0,464.947,541.723
2879,0.0,NaN,NaN
2880,0.0,NaN,NaN


In [82]:
# f_HDFStore["/tracks"].columns.levels[1].get_level_values(0)[1]
# print(dlc_hdf_info['individuals'])
# f_HDFStore['/tracks']
# f_HDFStore['/tracks'].get('scorer')

In [83]:
# scorer = f_HDFStore.get("tracks")
# indv = scorer.get("DLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500")
# bp = indv.get("mouse1")
# coords = bp.get("snout")
# coords

#f_HDFStore.append("DLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500/mouse1/snout/coords", frame_nr_df)

# coords.insert(0, "frame", list(range(1,1501)), True)
# bp.get("snout").insert(0, "frame", list(range(1,1501)), True)
# bp.get("snout")



In [84]:
# h5_df['DLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500', 'mouse1', KEYPOINT_WITH_TRANSPONDER]

In [85]:
dlc_hdf_info

{'scorer': Index(['DLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500'], dtype='object', name='scorer'),
 'individuals': Index(['mouse1', 'mouse2', 'mouse3'], dtype='object', name='individuals'),
 'bodyparts': Index(['leftEar', 'rightEar', 'shoulder', 'snout', 'spine1', 'spine2',
        'spine3', 'spine4', 'tail1', 'tail2', 'tailBase', 'tailEnd'],
       dtype='object', name='bodyparts'),
 'coords': Index(['likelihood', 'x', 'y'], dtype='object', name='coords')}

Second method of reading hdf files

In [86]:
dlc_hdf_info

{'scorer': Index(['DLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500'], dtype='object', name='scorer'),
 'individuals': Index(['mouse1', 'mouse2', 'mouse3'], dtype='object', name='individuals'),
 'bodyparts': Index(['leftEar', 'rightEar', 'shoulder', 'snout', 'spine1', 'spine2',
        'spine3', 'spine4', 'tail1', 'tail2', 'tailBase', 'tailEnd'],
       dtype='object', name='bodyparts'),
 'coords': Index(['likelihood', 'x', 'y'], dtype='object', name='coords')}

In [87]:

# tmp_h5 = h5_df['DLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500', 'mouse1', 'snout']
# # tmp_h5.loc[tmp_h5['coords'] == 0, 'x']
# tmp = tmp_h5.iloc[[0]]
# tmp

In [88]:
for mouse in dlc_hdf_info['individuals']:
    print(mouse)

mouse1
mouse2
mouse3


In [89]:
h5_df.close

NameError: name 'h5_df' is not defined

In [ ]:
f_HDFStore.get("tracks")

scorer      DLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500           \
individuals                                          mouse1            
bodyparts                                             snout            
coords                                                    x        y   
0                                                    90.367  469.270   
1                                                    92.883  478.379   
2                                                    95.490  483.816   
3                                                       NaN      NaN   
4                                                   114.148  491.662   
...                                                     ...      ...   
1495                                                748.706  348.446   
1496                                                733.987  354.947   
1497                                                722.757  365.944   
1498                                                708.796  376.132   
1499                                                698.765  376.551   

scorer                                                                 \
individuals                                                             
bodyparts              rightEar                      leftEar            
coords      likelihood        x        y likelihood        x        y   
0                1.000   94.806  411.087      0.993  124.526  446.288   
1                0.993   94.139  421.858      0.971  126.321  448.760   
2                1.000   97.946  420.932      0.675  127.621  447.286   
3                1.000  124.148  456.531      0.627  129.586  447.896   
4                1.000   88.194  442.456      0.999  129.483  451.786   
...                ...      ...      ...        ...      ...      ...   
1495             1.000  764.149  300.479      1.000  788.112  339.605   
1496             1.000  754.695  310.800      1.000  776.895  348.515   
1497             1.000  739.311  321.465      1.000  762.532  357.862   
1498             1.000  727.868  324.729      1.000  754.618  366.447   
1499             1.000  714.546  330.229      1.000  740.771  365.950   

scorer                           ...                                          \
individuals                      ...     mouse3                                
bodyparts              shoulder  ...   tailBase    tail1                       
coords      likelihood        x  ... likelihood        x        y likelihood   
0                  1.0  129.883  ...        1.0  335.042  175.385      1.000   
1                  1.0  130.779  ...        1.0  334.157  176.001      1.000   
2                  1.0  129.907  ...        1.0  334.783  175.254      1.000   
3                  1.0  128.957  ...        1.0  334.727  176.257      1.000   
4                  1.0  123.174  ...        1.0  335.553  176.094      1.000   
...                ...      ...  ...        ...      ...      ...        ...   
1495               1.0  796.508  ...        1.0  209.646  318.834      1.000   
1496               1.0  785.185  ...        1.0  211.987  317.409      1.000   
1497               1.0  774.877  ...        1.0  221.364  318.323      0.999   
1498               1.0  755.440  ...        1.0  219.333  310.596      1.000   
1499               1.0  749.343  ...        1.0  231.237  312.290      1.000   

scorer                                                                 
individuals                                                            
bodyparts      tail2                      tailEnd                      
coords             x        y likelihood        x        y likelihood  
0            375.061  187.306      0.998  441.128  190.861        1.0  
1            373.742  186.207      0.999  442.005  190.519        1.0  
2            374.576  187.110      0.998  441.244  190.812        1.0  
3            372.225  186.182      0.999  440.928  191.380        1.0  
4            372.496  186.506      0.996  442.

In [ ]:
f_HDFStore.get("tracks")[0]

KeyError: 0

In [ ]:
f.close()

NameError: name 'f' is not defined

In [ ]:
import h5py
h5py_path = r"results-recording_RP_2021_12_15--14-00-27_cut_first_min\after_analyzing\h5-test.h5"
f = h5py.File(h5py_path,'a')

In [ ]:
f

<HDF5 file "h5-test.h5" (mode r+)>

In [ ]:
f_items = (list(f.items()))
print("Items is base dir: ", f_items)

G = f.get("tracks")

# G_items = list(G.items())
# print("Items in the tracks group: ", G_items)

pd.DataFrame(G.get("table"))
#pd.DataFrame(G.get("table"))[0][0][1].shape

Items is base dir:  [('frame', <HDF5 group "/frame" (1 members)>), ('tracks', <HDF5 group "/tracks" (2 members)>)]


,0
0,"[0, [90.367, 469.27, 1.0, 94.806, 411.087, 0.9..."
1,"[1, [92.883, 478.379, 0.9929999709129333, 94.1..."
2,"[2, [95.49, 483.816, 1.0, 97.946, 420.932, 0.6..."
3,"[3, [nan, nan, 1.0, 124.148, 456.531, 0.626999..."
4,"[4, [114.148, 491.662, 1.0, 88.194, 442.456, 0..."
...,...
1495,"[1495, [748.706, 348.446, 1.0, 764.149, 300.47..."
1496,"[1496, [733.987, 354.947, 1.0, 754.695, 310.8,..."
1497,"[1497, [722.757, 365.944, 1.0, 739.311, 321.46..."
1498,"[1498, [708.796, 376.132, 1.0, 727.868, 324.72..."


In [ ]:
print(f.name)
print(list(f.keys()))
print(list(f.items()))
G_tracks = f.get("tracks")


/
['frame', 'tracks']
[('frame', <HDF5 group "/frame" (1 members)>), ('tracks', <HDF5 group "/tracks" (2 members)>)]


In [ ]:
list(f["tracks"].keys())

['_i_table', 'table']

In [ ]:
frame_grp = f.create_group("frame")

ValueError: Unable to create group (name already exists)

In [ ]:
frame_grp.create_dataset("frame_nr", range(1,4))

NameError: name 'frame_grp' is not defined

In [ ]:
f["frame"].name

'/frame'

In [ ]:
frame_grp["frame"] = list(range(1,1501))

NameError: name 'frame_grp' is not defined

In [ ]:
f.close()

In [ ]:
f_HDFStore = pd.HDFStore(h5py_path)
f_HDFStore.groups()

[/tracks (Group) ''
   children := ['table' (Table)]]

In [ ]:
f_HDFStore["tracks"]

scorer      DLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500           \
individuals                                          mouse1            
bodyparts                                             snout            
coords                                                    x        y   
0                                                    90.367  469.270   
1                                                    92.883  478.379   
2                                                    95.490  483.816   
3                                                       NaN      NaN   
4                                                   114.148  491.662   
...                                                     ...      ...   
1495                                                748.706  348.446   
1496                                                733.987  354.947   
1497                                                722.757  365.944   
1498                                                708.796  376.132   
1499                                                698.765  376.551   

scorer                                                                 \
individuals                                                             
bodyparts              rightEar                      leftEar            
coords      likelihood        x        y likelihood        x        y   
0                1.000   94.806  411.087      0.993  124.526  446.288   
1                0.993   94.139  421.858      0.971  126.321  448.760   
2                1.000   97.946  420.932      0.675  127.621  447.286   
3                1.000  124.148  456.531      0.627  129.586  447.896   
4                1.000   88.194  442.456      0.999  129.483  451.786   
...                ...      ...      ...        ...      ...      ...   
1495             1.000  764.149  300.479      1.000  788.112  339.605   
1496             1.000  754.695  310.800      1.000  776.895  348.515   
1497             1.000  739.311  321.465      1.000  762.532  357.862   
1498             1.000  727.868  324.729      1.000  754.618  366.447   
1499             1.000  714.546  330.229      1.000  740.771  365.950   

scorer                           ...                                          \
individuals                      ...     mouse3                                
bodyparts              shoulder  ...   tailBase    tail1                       
coords      likelihood        x  ... likelihood        x        y likelihood   
0                  1.0  129.883  ...        1.0  335.042  175.385      1.000   
1                  1.0  130.779  ...        1.0  334.157  176.001      1.000   
2                  1.0  129.907  ...        1.0  334.783  175.254      1.000   
3                  1.0  128.957  ...        1.0  334.727  176.257      1.000   
4                  1.0  123.174  ...        1.0  335.553  176.094      1.000   
...                ...      ...  ...        ...      ...      ...        ...   
1495               1.0  796.508  ...        1.0  209.646  318.834      1.000   
1496               1.0  785.185  ...        1.0  211.987  317.409      1.000   
1497               1.0  774.877  ...        1.0  221.364  318.323      0.999   
1498               1.0  755.440  ...        1.0  219.333  310.596      1.000   
1499               1.0  749.343  ...        1.0  231.237  312.290      1.000   

scorer                                                                 
individuals                                                            
bodyparts      tail2                      tailEnd                      
coords             x        y likelihood        x        y likelihood  
0            375.061  187.306      0.998  441.128  190.861        1.0  
1            373.742  186.207      0.999  442.005  190.519        1.0  
2            374.576  187.110      0.998  441.244  190.812        1.0  
3            372.225  186.182      0.999  440.928  191.380        1.0  
4            372.496  186.506      0.996  442.

In [ ]:
pickle_path = r"results-recording_RP_2021_12_15--14-00-27_cut_first_min\after_analyzing\recording_RP_2021_12_15--14-00-27_cut_first_minDLC_dlcrnetms5_DLC_SI_PiCamDec21shuffle1_102500_el.pickle"
pickle_file = pd.read_pickle(pickle_path)
pickle_file

{2: {'frame0000': array([[ 90.367     , 469.27      ,   1.        ,  -1.        ],
         [ 94.806     , 411.087     ,   0.99299997,  -1.        ],
         [124.526     , 446.288     ,   1.        ,  -1.        ],
         [129.883     , 416.077     ,   1.        ,  -1.        ],
         [144.059     , 412.76      ,   1.        ,  -1.        ],
         [161.208     , 415.871     ,   1.        ,  -1.        ],
         [169.376     , 416.115     ,   1.        ,  -1.        ],
         [185.137     , 414.431     ,   1.        ,  -1.        ],
         [197.49      , 403.775     ,   1.        ,  -1.        ],
         [225.119     , 371.68      ,   0.99900001,  -1.        ],
         [289.096     , 338.988     ,   0.99900001,  -1.        ],
         [         nan,          nan,   1.        ,          nan]]),
  'frame0001': array([[ 92.883     , 478.379     ,   0.99299997,  -1.        ],
         [ 94.139     , 421.858     ,   0.97100002,  -1.        ],
         [126.321     , 448.76 

In [ ]:
xleft_coord = 200
x_right_coord = 1180
y_up_coord = 225
y_buttom_coord = 500

x_RFID_coords = [i for j in (range(xleft_coord, x_right_coord, int((x_right_coord - xleft_coord) / 4)), range(xleft_coord, x_right_coord, int((x_right_coord - xleft_coord) / 4))) for i in j]
y_RFID_coords = [y_up_coord] * 4 + [y_buttom_coord] * 4

RFID_df_thermo_coords_adjusted = RFID_df_thermo_coords
RFID_df_thermo_coords_adjusted['IdLabel'] = x_RFID_coords
RFID_df_thermo_coords_adjusted['unitLabel'] = y_RFID_coords
RFID_df_thermo_coords_adjusted

In [ ]:
ygridline = [350]
#xgridlines = [i for i in range(100, 1300, int((1300-100)/5))]
xgridlines = [320, 570, 820]

fig = plt.figure()

image_path_name = '/Users/orabe/Desktop/h5/img00767.png'
im = plt.imread(image_path_name)
implot = plt.imshow(im)

ax = fig.gca()

ax.set_xticks(xgridlines)
ax.set_yticks(ygridline)

plt.scatter(x=RFID_df_thermo_coords.IdLabel, y=RFID_df_thermo_coords.unitLabel, marker='s', s=600, alpha=0.3, c='white')

for i, txt in enumerate(RFID_df_thermo_coords.IdRFID):
    ax.annotate(txt, (RFID_df_thermo_coords.IdLabel[i]-40, RFID_df_thermo_coords.unitLabel[i]+10))

plt.grid(color='r', linestyle='-', linewidth=1)
#plt.show()

plt.savefig("/Users/orabe/Desktop/h5/sample.png")

In [ ]:
image_path_name = '/Users/orabe/Desktop/h5/trajectorymouse2.png'
im = plt.imread(image_path_name)
implot = plt.imshow(im)

In [ ]:
# The plotting functions below are put here for simplicity and so that the user can edit them. Note that they 
# (or variants thereof) are in fact in standard DLC and accessible via:
# import deeplabcut

In [ ]:
# def get_cmap(n, name='hsv'):
#     return plt.cm.get_cmap(name, n)

# def Histogram(vector,color,bins):
#     dvector=np.diff(vector)
#     dvector=dvector[np.isfinite(dvector)]
#     plt.hist(dvector,color=color,histtype='step',bins=bins)

# def PlottingResults(Dataframe,bodyparts2plot,alphavalue=.2,pcutoff=.5,colormap='jet',fs=(4,3)):
#     ''' Plots poses vs time; pose x vs pose y; histogram of differences and likelihoods.'''
#     plt.figure(figsize=fs)
#     colors = get_cmap(len(bodyparts2plot),name = colormap)
#     scorer=Dataframe.columns.get_level_values(0)[0] #you can read out the header to get the scorer name!

#     for bpindex, bp in enumerate(bodyparts2plot):
#         Index=Dataframe[scorer][bp]['likelihood'].values > pcutoff
#         plt.plot(Dataframe[scorer][bp]['x'].values[Index],Dataframe[scorer][bp]['y'].values[Index],'.',color=colors(bpindex),alpha=alphavalue)

#     plt.gca().invert_yaxis()

#     sm = plt.cm.ScalarMappable(cmap=plt.get_cmap(colormap), norm=plt.Normalize(vmin=0, vmax=len(bodyparts2plot)-1))
#     sm._A = []
#     cbar = plt.colorbar(sm,ticks=range(len(bodyparts2plot)))
#     cbar.set_ticklabels(bodyparts2plot)
#     #plt.savefig(os.path.join(tmpfolder,"trajectory"+suffix))
#     plt.figure(figsize=fs)
#     Time=np.arange(np.size(Dataframe[scorer][bodyparts2plot[0]]['x'].values))

#     for bpindex, bp in enumerate(bodyparts2plot):
#         Index=Dataframe[scorer][bp]['likelihood'].values > pcutoff
#         plt.plot(Time[Index],Dataframe[scorer][bp]['x'].values[Index],'--',color=colors(bpindex),alpha=alphavalue)
#         plt.plot(Time[Index],Dataframe[scorer][bp]['y'].values[Index],'-',color=colors(bpindex),alpha=alphavalue)

#     sm = plt.cm.ScalarMappable(cmap=plt.get_cmap(colormap), norm=plt.Normalize(vmin=0, vmax=len(bodyparts2plot)-1))
#     sm._A = []
#     cbar = plt.colorbar(sm,ticks=range(len(bodyparts2plot)))
#     cbar.set_ticklabels(bodyparts2plot)
#     plt.xlabel('Frame index')
#     plt.ylabel('X and y-position in pixels')
#     #plt.savefig(os.path.join(tmpfolder,"plot"+suffix))

#     plt.figure(figsize=fs)
#     for bpindex, bp in enumerate(bodyparts2plot):
#         Index=Dataframe[scorer][bp]['likelihood'].values > pcutoff
#         plt.plot(Time,Dataframe[scorer][bp]['likelihood'].values,'-',color=colors(bpindex),alpha=alphavalue)

#     sm = plt.cm.ScalarMappable(cmap=plt.get_cmap(colormap), norm=plt.Normalize(vmin=0, vmax=len(bodyparts2plot)-1))
#     sm._A = []
#     cbar = plt.colorbar(sm,ticks=range(len(bodyparts2plot)))
#     cbar.set_ticklabels(bodyparts2plot)
#     plt.xlabel('Frame index')
#     plt.ylabel('likelihood')

#     #plt.savefig(os.path.join(tmpfolder,"plot-likelihood"+suffix))

#     plt.figure(figsize=fs)
#     bins=np.linspace(0,np.amax(Dataframe.max()),100)

#     for bpindex, bp in enumerate(bodyparts2plot):
#         Index=Dataframe[scorer][bp]['likelihood'].values < pcutoff
#         X=Dataframe[scorer][bp]['x'].values
#         X[Index]=np.nan
#         Histogram(X,colors(bpindex),bins)
#         Y=Dataframe[scorer][bp]['x'].values
#         Y[Index]=np.nan
#         Histogram(Y,colors(bpindex),bins)

#     sm = plt.cm.ScalarMappable(cmap=plt.get_cmap(colormap), norm=plt.Normalize(vmin=0, vmax=len(bodyparts2plot)-1))
#     sm._A = []
#     cbar = plt.colorbar(sm,ticks=range(len(bodyparts2plot)))
#     cbar.set_ticklabels(bodyparts2plot)
#     plt.ylabel('Count')
#     plt.xlabel('DeltaX and DeltaY')
    
#     #plt.savefig(os.path.join(tmpfolder,"hist"+suffix))

In [ ]:
# bodyparts=Dataframe.columns.get_level_values(1) #you can read out the header to get body part names!

# bodyparts2plot=bodyparts #you could also take a subset, i.e. =['snout']

# %matplotlib inline
# PlottingResults(Dataframe,bodyparts2plot,alphavalue=.2,pcutoff=.5,fs=(8,4))

# # These plots can are interactive and can be customized (see https://matplotlib.org/) [in the code above]
# # note that the snout and other bpts jitter in this example that was not trained for long.